To run this notebook just press _"Run All"_ <span style="opacity:.8;">(in Google Colab: <b>Runtime ▸ Run all</b>)</span>

<p align="center">
  <a href="https://docs.fenic.ai">Read the Docs</a> •
  <a href="https://discord.com/invite/GdqF3J7huR">Join Discord</a> •
  <a href="https://github.com/typedef-ai/fenic">⭐️ Star fenic</a>
</p>

To install fenic locally, just follow the instructions on the [Github Repo](https://github.com/typedef-ai/fenic)

You will learn how to work with transcripts in fenic using its native transcript processing capabilities, including format detection, parsing, and semantic extraction of structured information from unstructured meeting content.

If this notebook helps, please give <a href="https://github.com/typedef-ai/fenic" target="_blank" rel="noopener noreferrer">fenic</a> a ⭐️ — it really helps!

## Installation

In [ ]:
!pip uninstall -y sklearn-compat ibis-framework imbalanced-learn
!pip install fenic matplotlib seaborn polars==1.30.0

In [ ]:
import os
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [ ]:
from pydantic import BaseModel, Field

from typing import List, Optional

import fenic as fc

## Sample Meeting Transcripts

Let's define three sample meeting transcripts representing different types of engineering meetings:

In [ ]:
# Engineering Architecture Review Transcript
architecture_review_transcript = """Sarah (00:02:15)
  Alright, so we're here to discuss the user service redesign. Um, Mike can you walk us through the current bottlenecks?

Mike (00:02:28)
  Sure, so right now we're seeing about 2-second response times on the /users endpoint during peak hours. The main issue is we're hitting the PostgreSQL database directly for every request.

David (00:02:45)
  Right, and I think we discussed adding Redis caching last sprint but didn't get to it.

Sarah (00:03:01)
  OK so action item for Mike - investigate Redis implementation. What about the authentication service dependency?

Mike (00:03:18)
  Yeah that's another bottleneck. Every user request has to validate the JWT token with the auth service. We could cache those validations too.

David (00:03:35)
  Actually, we had that incident last week where auth service went down and took out the whole user flow. Incident #INC-2024-007.

Sarah (00:03:48)
  Good point. So we need resilience there. Mike, can you also look into circuit breaker patterns? I'm thinking we implement that by end of Q1.

Mike (00:04:05)
  Yep, I'll research both Redis caching and circuit breakers. Should have a design doc ready by next Friday.

Sarah (00:04:15)
  Perfect. David, anything else on the database side?

David (00:04:22)
  We should consider read replicas too. I've been seeing high CPU on the primary during reports generation.

Sarah (00:04:35)
  OK, let's add that to the backlog. Decision: we're moving forward with the caching and circuit breaker approach for user service optimization."""

In [ ]:
# Incident Post-Mortem Transcript
incident_postmortem_transcript = """Alex (00:01:05)
  OK everyone, this is the post-mortem for yesterday's outage. Incident #INC-2024-012. We had approximately 45 minutes of downtime starting at 14:30 UTC.

Jordan (00:01:20)
  The root cause was the payment service running out of memory. We saw heap size spike to 8GB before the JVM crashed.

Sam (00:01:35)
  Yeah, I was monitoring the dashboards. CPU was normal but memory kept climbing. No garbage collection could keep up.

Alex (00:01:48)
  What triggered it? Any recent deployments?

Jordan (00:01:55)
  We deployed the new batch processing feature Tuesday morning. I think there's a memory leak in the transaction processing loop.

Sam (00:02:10)
  Action item for me - I'll review the batch processing code and look for leaked objects or unclosed resources.

Alex (00:02:20)
  Good. Jordan, can you increase the heap size as a temporary mitigation? Maybe 12GB instead of 8?

Jordan (00:02:30)
  Already done. I bumped it to 12GB and added memory alerts at 80% usage.

Alex (00:02:40)
  Perfect. Sam, when can you have the code review done?

Sam (00:02:45)
  I'll have findings by tomorrow EOD. If it's a simple fix, we can hotfix Friday.

Alex (00:02:55)
  Decision: temporary mitigation in place, code review by Thursday, hotfix target Friday if feasible."""

In [ ]:
# Sprint Planning Transcript
sprint_planning_transcript = """Emma (00:00:30)
  Alright team, let's plan Sprint 23. We have 40 story points available this sprint. What's our priorities?

Ryan (00:00:45)
  The user authentication refactor is our biggest priority. That's been blocking the mobile team for two weeks.

Lisa (00:00:58)
  Right, and we estimated that at 13 story points. We also need to address the API rate limiting issues.

Emma (00:01:12)
  Good point. Ryan, can you take the auth refactor? And Lisa, would you handle the rate limiting?

Ryan (00:01:20)
  Yeah, I can do the auth work. I'll need to coordinate with the mobile team on the JWT token format changes.

Lisa (00:01:35)
  Sure, I'll take rate limiting. I'm thinking we implement token bucket algorithm with Redis backend.

Emma (00:01:50)
  Perfect. What about the database migration for the user profiles table?

Ryan (00:02:05)
  That's risky. We're adding three new columns and need to backfill data for 2 million users.

Lisa (00:02:18)
  Action item - let's create a migration plan with zero-downtime strategy. I can draft that by Wednesday.

Emma (00:02:35)
  Great. Decision: Sprint 23 priorities are auth refactor, rate limiting, and migration planning. Ryan and Lisa are the leads."""

## Configure Session and Create DataFrame

Let's set up a fenic session with semantic capabilities and create a DataFrame with our meeting transcripts:

In [ ]:
# Configure session with semantic capabilities
config = fc.SessionConfig(
    app_name="meeting_transcript_processing",
    semantic=fc.SemanticConfig(
        language_models={
            "mini": fc.OpenAILanguageModel(model_name="gpt-4o-mini", rpm=500, tpm=200_000)
        }
    ),
)

# Create session
session = fc.Session.get_or_create(config)

# Create DataFrame with meeting transcripts
transcripts_data = [
    {
        "meeting_id": "ARCH-2024-001",
        "meeting_type": "Architecture Review",
        "transcript": architecture_review_transcript,
    },
    {
        "meeting_id": "INC-2024-012",
        "meeting_type": "Incident Post-Mortem",
        "transcript": incident_postmortem_transcript,
    },
    {
        "meeting_id": "SPRINT-23",
        "meeting_type": "Sprint Planning",
        "transcript": sprint_planning_transcript,
    },
]

transcripts_df = session.create_dataframe(transcripts_data)

print("Meeting transcripts loaded:")
transcripts_df.select(fc.col("meeting_id"), fc.col("meeting_type")).show()

## Step 1: Native Transcript Parsing

Fenic provides built-in transcript processing capabilities. Let's parse the transcripts:

In [ ]:
# Parse transcripts into structured format
# generic is a commonly found format for transcripts that follows the format:
# speaker (00:00:00)
# content
# speaker (00:00:00)
parsed_transcripts_df = transcripts_df.with_column(
    "structured_transcript",
    fc.text.parse_transcript(fc.col("transcript"), "generic"),
)

print("Parsed transcript structure sample:")
sample_parsed = parsed_transcripts_df.select(
    fc.col("meeting_id"), fc.col("structured_transcript")
).limit(1)
sample_parsed.show()

## Step 2: Extract Individual Speaking Segments

Now let's explode the structured transcript data into individual speaking segments for analysis:

In [ ]:
# Explode structured transcript into individual segments
segments_df = (
    parsed_transcripts_df.explode("structured_transcript")
    .unnest("structured_transcript")
    .select(
        fc.col("meeting_id"),
        fc.col("meeting_type"),
        fc.col("speaker"),
        fc.col("start_time"),
        fc.col("content"),
    )
)

print("Individual speaking segments:")
segments_df.show(5)

## Step 3: Define Semantic Extraction Schemas

Let's define schemas to extract structured information from the unstructured transcript content:

In [ ]:
class TechnicalEntitiesSchema(BaseModel):
    """Schema for extracting technical entities from meeting transcript segments."""

    services: List[str] = Field(
        description="Technical services or systems mentioned (e.g., user-service, auth-service, payment-service)"
    )
    technologies: List[str] = Field(
        description="Technologies, databases, or tools mentioned (e.g., Redis, PostgreSQL, JWT, JVM)"
    )
    metrics: List[str] = Field(
        description="Performance metrics, numbers, or measurements mentioned "
                    "(e.g., response times, memory usage)"
    )
    incident_references: List[str] = Field(
        description="Incident IDs, ticket numbers, or reference numbers mentioned"
    )


# Action items schema
class ActionItemSchema(BaseModel):
    """Schema for extracting action items from meeting transcript segments."""

    has_action_item: str = Field(
        description="Whether this segment contains an action item (yes/no)"
    )
    assignee: Optional[str] = Field(
        default=None, description="Person assigned to the action item (if any)"
    )
    task_description: str = Field(
        description="Description of the task or action to be completed"
    )
    deadline: Optional[str] = Field(
        default=None, description="When the task should be completed (if mentioned)"
    )


# Decisions schema
class DecisionSchema(BaseModel):
    """Schema for extracting decisions from meeting transcript segments."""

    has_decision: str = Field(
        description="Whether this segment contains a decision (yes/no)"
    )
    decision_summary: str = Field(description="Summary of the decision made")
    decision_rationale: Optional[str] = Field(
        default=None, description="Why this decision was made (if mentioned)"
    )


print("Created schemas for:")
print("- Technical entities (services, technologies, metrics, incidents)")
print("- Action items (assignee, task, deadline)")
print("- Decisions (summary, rationale)")

## Step 4: Apply Semantic Extraction

Now let's apply semantic extraction to each segment to extract technical entities, action items, and decisions:

In [ ]:
# Extract technical entities, action items, and decisions from each segment
enriched_df = (
    segments_df.with_column(
        "technical_entities",
        fc.semantic.extract(fc.col("content"), TechnicalEntitiesSchema),
    )
    .with_column(
        "action_items", fc.semantic.extract(fc.col("content"), ActionItemSchema)
    )
    .with_column("decisions", fc.semantic.extract(fc.col("content"), DecisionSchema))
    .cache()
)

print("Applied semantic extraction to all segments")

## Step 5: Extract and Structure Insights

Let's unnest the extracted data to create a structured insights DataFrame:

In [ ]:
# Unnest extracted data and create structured insights
insights_df = (
    enriched_df.unnest("technical_entities")
    .unnest("action_items")
    .unnest("decisions")
    .select(
        fc.col("meeting_id"),
        fc.col("meeting_type"),
        fc.col("speaker"),
        fc.col("start_time"),
        fc.col("content"),
        fc.col("services"),
        fc.col("technologies"),
        fc.col("metrics"),
        fc.col("incident_references"),
        fc.col("has_action_item"),
        fc.col("assignee"),
        fc.col("task_description"),
        fc.col("deadline"),
        fc.col("has_decision"),
        fc.col("decision_summary"),
        fc.col("decision_rationale"),
    )
)

print("Structured insights sample:")
insights_df.select("*").show(5)

## Step 6: Meeting-Level Analytics

Now let's aggregate the insights to generate meeting-level analytics:

In [ ]:
# Extract action items summary
action_items_summary = insights_df.filter(fc.col("has_action_item") == "yes").select(
    fc.col("meeting_id"),
    fc.col("meeting_type"),
    fc.col("assignee"),
    fc.col("task_description"),
    fc.col("deadline"),
)
print("Action Items Summary:")
action_items_summary.show()

In [ ]:
# Extract decisions summary
decisions_summary = insights_df.filter(fc.col("has_decision") == "yes").select(
    fc.col("meeting_id"),
    fc.col("meeting_type"),
    fc.col("decision_summary"),
    fc.col("decision_rationale"),
)

print("Decisions Summary:")
decisions_summary.show()

In [ ]:
# Analyze most mentioned services
all_services = (
    insights_df.select(fc.col("meeting_id"), fc.col("services"))
    .explode("services")
    .filter(fc.col("services").is_not_null() & (fc.col("services") != ""))
    .group_by("services")
    .agg(fc.count(fc.col("meeting_id")).alias("mention_count"))
    .sort("mention_count", ascending=False)
)

print("Most Mentioned Services:")
all_services.show()

In [ ]:
# Analyze most mentioned technologies
all_technologies = (
    insights_df.select(fc.col("meeting_id"), fc.col("technologies"))
    .explode("technologies")
    .filter(fc.col("technologies").is_not_null() & (fc.col("technologies") != ""))
    .group_by("technologies")
    .agg(fc.count(fc.col("meeting_id")).alias("mention_count"))
    .sort("mention_count", ascending=False)
)

print("Most Mentioned Technologies:")
all_technologies.show()

In [ ]:
# Calculate meeting productivity metrics
meeting_stats = insights_df.group_by("meeting_id", "meeting_type").agg(
    fc.count(fc.col("speaker")).alias("total_segments"),
    fc.sum(
        fc.when(fc.col("has_action_item") == "yes", fc.lit(1)).otherwise(fc.lit(0))
    ).alias("action_items_count"),
    fc.sum(
        fc.when(fc.col("has_decision") == "yes", fc.lit(1)).otherwise(fc.lit(0))
    ).alias("decisions_count"),
    fc.sum(
        fc.when(fc.col("services").is_not_null(), fc.lit(1)).otherwise(fc.lit(0))
    ).alias("technical_mentions"),
)

print("Meeting Productivity Metrics:")
meeting_stats.show()

## Step 7: Generate Actionable Outputs

Finally, let's generate actionable insights from our analysis:

In [ ]:
# Count unique assignees and their workload
assignee_workload = (
    insights_df.filter(fc.col("has_action_item") == "yes")
    .group_by("assignee")
    .agg(fc.count("*").alias("assigned_tasks"))
    .order_by(fc.col("assigned_tasks").desc())
)

print("Team Member Workload (Action Items):")
assignee_workload.show()

In [ ]:
# Timeline of action items
action_timeline = (
    insights_df.filter(
        (fc.col("has_action_item") == "yes") & (fc.col("deadline").is_not_null())
    )
    .select(
        fc.col("meeting_id"),
        fc.col("assignee"),
        fc.col("task_description"),
        fc.col("deadline"),
    )
    .order_by("deadline")
)

print("Action Items Timeline:")
action_timeline.show()

## Next Steps

A few other examples to explore:

- Extract structured metadata from unstructured text [notebook](https://colab.research.google.com/github/typedef-ai/fenic/blob/main/examples/document_extraction/document_extraction.ipynb)
- Named Entity Recognition [notebook](https://colab.research.google.com/github/typedef-ai/fenic/blob/main/examples/named_entity_recognition/ner.ipynb)
- Classification of new articles for article bias detection [notebook](https://colab.research.google.com/github/typedef-ai/fenic/blob/main/examples/news_analysis/news_analysis.ipynb)

---

<p align="center" style="margin:18px 0 6px; font-size:1.05em;">
  Enjoyed this? Help others find it.
</p>

<p align="center" style="margin:0 0 12px;">
<a href="https://github.com/typedef-ai/fenic">⭐️ Give fenic a Star ⭐️</a>
</p>